# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 21 2022 2:50PM,246983,20,8555948-T,"ACI Healthcare USA, Inc.",Released
1,Sep 21 2022 2:45PM,246982,10,MSP214193,"Methapharm, Inc.",Released
2,Sep 21 2022 2:44PM,246981,10,8555951,Yusen – 3D Matrix,Executing
3,Sep 21 2022 2:39PM,246978,19,60017422,"GUSA Granules USA, Inc.",Released
4,Sep 21 2022 2:33PM,246977,10,0086000299,ISDIN Corporation,Released
5,Sep 21 2022 2:33PM,246977,10,0086000308,ISDIN Corporation,Released
6,Sep 21 2022 2:33PM,246977,10,0086000310,ISDIN Corporation,Released
7,Sep 21 2022 2:33PM,246977,10,0086000313,ISDIN Corporation,Released
8,Sep 21 2022 2:33PM,246977,10,0086000316,ISDIN Corporation,Released
9,Sep 21 2022 2:33PM,246977,10,0086000315,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
36,246977,Released,13
37,246978,Released,1
38,246981,Executing,1
39,246982,Released,1
40,246983,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
246977,NaN,NaN,13.0
246978,NaN,NaN,1.0
246981,NaN,1.0,NaN
246982,NaN,NaN,1.0
246983,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246872,0.0,0.0,1.0
246878,0.0,1.0,0.0
246887,0.0,0.0,1.0
246904,0.0,3.0,3.0
246905,0.0,0.0,19.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246872,0,0,1
246878,0,1,0
246887,0,0,1
246904,0,3,3
246905,0,0,19


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246872,0,0,1
1,246878,0,1,0
2,246887,0,0,1
3,246904,0,3,3
4,246905,0,0,19


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246872,,,1
1,246878,,1,
2,246887,,,1
3,246904,,3,3
4,246905,,,19


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 21 2022 2:50PM,246983,20,"ACI Healthcare USA, Inc."
1,Sep 21 2022 2:45PM,246982,10,"Methapharm, Inc."
2,Sep 21 2022 2:44PM,246981,10,Yusen – 3D Matrix
3,Sep 21 2022 2:39PM,246978,19,"GUSA Granules USA, Inc."
4,Sep 21 2022 2:33PM,246977,10,ISDIN Corporation
17,Sep 21 2022 2:26PM,246974,10,ISDIN Corporation
50,Sep 21 2022 2:24PM,246975,10,Emerginnova
55,Sep 21 2022 2:15PM,246973,16,Sartorius Stedim North America
56,Sep 21 2022 2:11PM,246971,20,"ACI Healthcare USA, Inc."
70,Sep 21 2022 2:07PM,246969,10,"Citieffe, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 21 2022 2:50PM,246983,20,"ACI Healthcare USA, Inc.",,,1
1,Sep 21 2022 2:45PM,246982,10,"Methapharm, Inc.",,,1
2,Sep 21 2022 2:44PM,246981,10,Yusen – 3D Matrix,,1,
3,Sep 21 2022 2:39PM,246978,19,"GUSA Granules USA, Inc.",,,1
4,Sep 21 2022 2:33PM,246977,10,ISDIN Corporation,,,13
5,Sep 21 2022 2:26PM,246974,10,ISDIN Corporation,,,33
6,Sep 21 2022 2:24PM,246975,10,Emerginnova,,,5
7,Sep 21 2022 2:15PM,246973,16,Sartorius Stedim North America,,1,
8,Sep 21 2022 2:11PM,246971,20,"ACI Healthcare USA, Inc.",,4,10
9,Sep 21 2022 2:07PM,246969,10,"Citieffe, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 21 2022 2:50PM,246983,20,"ACI Healthcare USA, Inc.",1,,
1,Sep 21 2022 2:45PM,246982,10,"Methapharm, Inc.",1,,
2,Sep 21 2022 2:44PM,246981,10,Yusen – 3D Matrix,,1,
3,Sep 21 2022 2:39PM,246978,19,"GUSA Granules USA, Inc.",1,,
4,Sep 21 2022 2:33PM,246977,10,ISDIN Corporation,13,,
5,Sep 21 2022 2:26PM,246974,10,ISDIN Corporation,33,,
6,Sep 21 2022 2:24PM,246975,10,Emerginnova,5,,
7,Sep 21 2022 2:15PM,246973,16,Sartorius Stedim North America,,1,
8,Sep 21 2022 2:11PM,246971,20,"ACI Healthcare USA, Inc.",10,4,
9,Sep 21 2022 2:07PM,246969,10,"Citieffe, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 21 2022 2:50PM,246983,20,"ACI Healthcare USA, Inc.",1,,
1,Sep 21 2022 2:45PM,246982,10,"Methapharm, Inc.",1,,
2,Sep 21 2022 2:44PM,246981,10,Yusen – 3D Matrix,,1,
3,Sep 21 2022 2:39PM,246978,19,"GUSA Granules USA, Inc.",1,,
4,Sep 21 2022 2:33PM,246977,10,ISDIN Corporation,13,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 21 2022 2:50PM,246983,20,"ACI Healthcare USA, Inc.",1.0,NaN,NaN
1,Sep 21 2022 2:45PM,246982,10,"Methapharm, Inc.",1.0,NaN,NaN
2,Sep 21 2022 2:44PM,246981,10,Yusen – 3D Matrix,NaN,1.0,NaN
3,Sep 21 2022 2:39PM,246978,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN
4,Sep 21 2022 2:33PM,246977,10,ISDIN Corporation,13.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 21 2022 2:50PM,246983,20,"ACI Healthcare USA, Inc.",1.0,0.0,0.0
1,Sep 21 2022 2:45PM,246982,10,"Methapharm, Inc.",1.0,0.0,0.0
2,Sep 21 2022 2:44PM,246981,10,Yusen – 3D Matrix,0.0,1.0,0.0
3,Sep 21 2022 2:39PM,246978,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0
4,Sep 21 2022 2:33PM,246977,10,ISDIN Corporation,13.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3704198,103.0,21.0,3.0
12,493811,2.0,0.0,0.0
15,493855,10.0,0.0,18.0
16,493881,0.0,6.0,0.0
19,1234793,28.0,8.0,0.0
20,1234706,16.0,7.0,0.0
21,246878,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3704198,103.0,21.0,3.0
1,12,493811,2.0,0.0,0.0
2,15,493855,10.0,0.0,18.0
3,16,493881,0.0,6.0,0.0
4,19,1234793,28.0,8.0,0.0
5,20,1234706,16.0,7.0,0.0
6,21,246878,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,103.0,21.0,3.0
1,12,2.0,0.0,0.0
2,15,10.0,0.0,18.0
3,16,0.0,6.0,0.0
4,19,28.0,8.0,0.0
5,20,16.0,7.0,0.0
6,21,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,103.0
1,12,Released,2.0
2,15,Released,10.0
3,16,Released,0.0
4,19,Released,28.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,3.0,0.0,18.0,0.0,0.0,0.0,0.0
Executing,21.0,0.0,0.0,6.0,8.0,7.0,1.0
Released,103.0,2.0,10.0,0.0,28.0,16.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,3.0,0.0,18.0,0.0,0.0,0.0,0.0
1,Executing,21.0,0.0,0.0,6.0,8.0,7.0,1.0
2,Released,103.0,2.0,10.0,0.0,28.0,16.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,3.0,0.0,18.0,0.0,0.0,0.0,0.0
1,Executing,21.0,0.0,0.0,6.0,8.0,7.0,1.0
2,Released,103.0,2.0,10.0,0.0,28.0,16.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()